This file cleans the raw json data and adds the price range column. The Output is the filteredprice.csv.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
from sklearn.neighbors import KDTree
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
print(os.getcwd())
os.chdir("/content/gdrive/My Drive/Colab Notebooks (1)/yelp_dataset")
print(os.getcwd())
def extract_price_range(attributes):
    if attributes and "RestaurantsPriceRange2" in attributes:
        price_range = attributes["RestaurantsPriceRange2"]
        if price_range == 'None':
          return None
        return price_range
    return None

Mounted at /content/gdrive
/content
/content/gdrive/.shortcut-targets-by-id/1m5dVPIYDUmApGRmzUSFDadtixbOm-Lu2/Colab Notebooks/yelp_dataset


In [ ]:
#Data Cleaning
#Load Data
filename = "yelp_academic_dataset_business.json"
df = pd.read_json(filename, lines = True)
#Is Open filter
open_df = df[df['is_open']==1]
#Restaurant filter
open_df['incl'] = pd.Series(open_df['categories']).str.contains('Restaurants').tolist()
res_df = open_df[open_df['incl']== True]
#Drop Some Columns
res_df = res_df.drop(['incl', 'hours', 'is_open'], axis =1)
#Add Price Range Data
res_df['price_range'] = res_df['attributes'].apply(extract_price_range)
res_df

<ipython-input-2-a12ec5c28988>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  open_df['incl'] = pd.Series(open_df['categories']).str.contains('Restaurants').tolist()


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,price_range
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",1
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...",1
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",1
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks",None
12,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,"{'RestaurantsReservations': 'False', 'Restaura...","American (Traditional), Restaurants, Diners, B...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150323,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,246 N 8th St,Boise,ID,83702,43.616590,-116.202383,4.5,998,"{'BikeParking': 'True', 'Alcohol': 'u'full_bar...","Bars, Gastropubs, Sandwiches, Nightlife, Resta...",2
150325,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,19 N Bishop Ave,Clifton Heights,PA,19018,39.925656,-75.310344,3.0,11,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Restaurants, Sandwiches, Convenience Stores, C...",1
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,"{'WiFi': ''free'', 'RestaurantsGoodForGroups':...","Cafes, Juice Bars & Smoothies, Coffee & Tea, R...",1
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,"{'WheelchairAccessible': 'False', 'Restaurants...","Restaurants, Mexican",2


In [ ]:
1-(res_df['price_range'].isna().sum()/34987)

0.8237059479235145

In [ ]:
#Creating a dictionary that has the counts of all the different categories found in the dataset
categories_list= []
categories_dict = {}
i = 0
for i in range(len(res_df)):
  new = res_df.categories.iloc[i]
  new = list(new.split(","))
  for n,s in enumerate(new):
    new[n] = s.strip()

  old = categories_list
  categories_list =  old + new

for c in range(len(categories_list)):
  categories_dict[categories_list[c]]= categories_list.count(categories_list[c])
#print(categories_dict)

#Sorting and take the top 20 (excluding food and restaurant)
sorted_categories = sorted(categories_dict.items(), key=lambda x:x[1], reverse= True)

new_cols = []
for i in range(2,23):
  new_cols.append(sorted_categories[i][0])
new_cols

['Sandwiches',
 'Nightlife',
 'Bars',
 'American (Traditional)',
 'Fast Food',
 'Pizza',
 'Breakfast & Brunch',
 'Burgers',
 'American (New)',
 'Mexican',
 'Coffee & Tea',
 'Italian',
 'Seafood',
 'Chicken Wings',
 'Chinese',
 'Event Planning & Services',
 'Salad',
 'Delis',
 'Cafes',
 'Caterers',
 'Specialty Food']

In [ ]:
def bincol(string):
  tf = [string in i for i in res_df.categories]
  res_df[string] = tf
for string in new_cols:
  bincol(string)
res_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Italian,Seafood,Chicken Wings,Chinese,Event Planning & Services,Salad,Delis,Cafes,Caterers,Specialty Food
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,False,False,False,False,False,False,False,False,False,False
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,False,False,False,False,False,False,False,False,False,False
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,...,False,False,False,False,False,False,False,False,False,False
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,...,False,False,False,False,False,False,False,False,False,False
12,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150323,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,246 N 8th St,Boise,ID,83702,43.616590,-116.202383,4.5,998,...,False,False,False,False,False,False,False,False,False,False
150325,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,19 N Bishop Ave,Clifton Heights,PA,19018,39.925656,-75.310344,3.0,11,...,False,False,False,False,False,False,False,False,False,False
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,...,False,False,False,False,False,False,False,True,False,False
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
res_def

In [ ]:
res_df.to_csv("filteredprice.csv", index=False)